<a href="https://colab.research.google.com/github/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning/blob/main/GB886_VII_13_PanelExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Panel Data Example

We will consider a simple example to illustrate some panel data ideas.

We wull use 'linearmodels' which is a Python library that extends statsmodels with Panel regression and other functionality. We need to install it:

In [ ]:
!pip install linearmodels

Let's load the relevant libaries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from sklearn.metrics import mean_absolute_error, mean_squared_error


And next, let's load our data. We consider an example where we have Sales data from an office supplier from different clients over five years, and information on Company Size and the company-specific growth rate in the prior year. The idea is that larger companies and companies that grow more will order more.

In [ ]:
!git clone https://github.com/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning.git
data = pd.read_csv('MSDIA_PredictiveModelingAndMachineLearning/GB886_VII_13_SalesPanel.csv')
data.head()

Let's explore the data a litte, to understand the distribution and relationship between variables:

In [ ]:
print(data.describe())

plt.figure(figsize=(10,6))
sns.histplot(data['Sales'], bins=30, kde=True)
plt.title('Distribution of Sales')
plt.xlabel('Sales')
plt.ylabel('Frequency')
plt.show()

Let's run two models, a pooled OLS (POLS) model and our panel model, where we include company specific effects. Let's define our feature matrix and the target:

In [ ]:
my_data = data.set_index(['Client', 'Year'])

features = ['Company_Size', 'Growth_Rate']
target = 'Sales'

X = my_data[features]
y = my_data[target]

### POLS Model

Let's first run the pooled model where we ignore the panel structure:

In [ ]:
# Add a constant to the model (intercept)
X_pols = sm.add_constant(X)

# Fit the Pooled OLS model
pols_model = sm.OLS(y, X_pols).fit()

# Summary of the POLS model
print(pols_model.summary())

Let's calculate the mean-squared error:

In [ ]:
np.sqrt(pols_model.mse_resid)

### Fixed Effects Models

Let's now run the panel model with fixed effects that includes constant terms for each of the entities


In [ ]:
# Fit the Fixed Effects model
fe_model = PanelOLS(y,X, entity_effects=True).fit(cov_type='clustered', cluster_entity=True)

# Summary of the FE model
print(fe_model.summary)

We notice the coefficients change a bit.

Let's calculate the RMSE:

In [ ]:
np.sqrt(fe_model.resid_ss/250)

It is quite a bit lower. The reason is that we have company fixed effects, so essentially company specific intercepts!

Let's look at them:

In [ ]:
fe_model.estimated_effects

In [ ]:
fixed_effects = fe_model.estimated_effects.unstack(fill_value=0)

plt.figure(figsize=(10, 6))
sns.boxplot(data=fixed_effects)
plt.title('Distribution of Fixed Effects by Entity')
plt.xlabel('Entity')
plt.ylabel('Fixed Effect')
plt.xticks(rotation=90)
plt.show()

So, the idea is that we exploit the information that we have that these data come from different clients. We can use them to "explain" some of the variation, so that the unexplained part of the model is smaller:

In [ ]:
# Residuals from POLS model
residuals_pols = pols_model.resid

# Residuals from FE model
residuals_fe = fe_model.resids

# Plotting the residuals
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.histplot(residuals_pols, kde=True)
plt.title('Distribution of Residuals (POLS)')
plt.xlabel('Residuals')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
sns.histplot(residuals_fe, kde=True)
plt.title('Distribution of Residuals (FE)')
plt.xlabel('Residuals')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Summary statistics of residuals
print("POLS Residuals Summary:")
print(residuals_pols.describe())

print("\nFE Residuals Summary:")
print(residuals_fe.describe())
